# TP1 - Introdução a Banco de Dados - 2019/2
**Lucas Fonseca Mundim - 2015042134**

---

In [2]:
import pandas as pd
import io
import sqlite3 as sql
import os

In [3]:
connection = sql.connect('tp1.db')

### Executar apenas na primeira iteração:

In [ ]:
cursor = connection.cursor()
data = io.open('despesas_publicas_tp1.sql', 'r', encoding='ISO-8859-1')
database = data.read()
cursor.executescript(database)

In [4]:
def query(query_string):
    return pd.read_sql_query(query_string, connection)

Matrícula: 2015**042134**

Exercícios: 1,5,3,2,4,6 <- Reordenados para 1,2,3,4,5,6
#### Questão 1
Liste todos os códigos e nomes de funções da subfunção que possui o maior número de funções.

In [9]:
# Recover the sub-function with most functions
first = query('select cod_funcao, cod_subfuncao, count(cod_subfuncao) as count from detalhamento_funcao group by cod_subfuncao limit 1')
# Recover the functions belonging to the found sub-function
second = query('select cod_funcao from detalhamento_funcao where cod_subfuncao=122')
# Recover the final code and name for the wanted functions
third = query('select * from funcao where cod_funcao in (5,6,26)')

print(first)
print(second)
print(third)

# All at once...
q1_last = ('select funcao.nome_funcao, funcao.cod_funcao, detalhamento_funcao.cod_subfuncao ' + 
'from funcao ' +
'natural join detalhamento_funcao ' +
'where detalhamento_funcao.cod_subfuncao=(' +
'select cod_subfuncao from detalhamento_funcao group by cod_subfuncao limit 1)')
    
q1 = query(q1_last)
q1

   COD_FUNCAO  COD_SUBFUNCAO  count
0           5            122      3
   COD_FUNCAO
0           5
1           6
2          26
   COD_FUNCAO        NOME_FUNCAO
0           5    Defesa nacional
1           6  Segurança pública
2          26         Transporte


,NOME_FUNCAO,COD_FUNCAO,COD_SUBFUNCAO
0,Defesa nacional,5,122
1,Segurança pública,6,122
2,Transporte,26,122


#### Questão 2
Liste a média de despesas públicas por função (nome) e subfunção (nome), apenas nos casos em que a média excedeu mil reais.

In [10]:
q2_query = ('select * from('
    'select ' +
    'funcao.nome_funcao, ' + 
    'subfuncao.nome_subfuncao, ' +
    'avg(valor_despesa) as media ' +
    'from despesa_publica ' +
    'natural join detalhamento_funcao ' +
    'natural join funcao ' +
    'natural join subfuncao ' +
    'group by nome_funcao, nome_subfuncao ' +
    'order by media desc ' +
    ') where media >= 1000')
q2 = query(q2_query)
q2

,nome_funcao,nome_subfuncao,media
0,Educação,Educação básica,4398.185000
1,Assistência social,Tecnologia da informação,2266.087692
2,Educação,Ensino profissional,1826.135152
3,Assistência social,Assistência comunitária,1713.411805


#### Questão 3
Liste todos os nomes dos programas de governo e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [11]:
q3_query = ('select ' +
    'nome_programa_governo, sum(valor_despesa) as despesas_totais ' +
    'from programa_governo ' +
    'natural join (select * from despesa_publica where ano_despesa=2018) ' +
    'group by cod_programa_governo order by despesas_totais desc')
q3 = query(q3_query)
q3

,NOME_PROGRAMA_GOVERNO,despesas_totais
0,BRASIL SEM MISERIA,387175.60
1,ENEM,334262.06
2,PRONATEC,60262.46
3,CONTROLE DO CANCER,38971.19
4,MAIS MEDICOS,14400.00
5,REGULARIZACAO FUNDIARIA,1497.26
6,REDE CEGONHA,1210.00
7,VIVER SEM LIMITE,0.00
8,SOS EMERGENCIA,0.00
9,MELHOR EM CASA,0.00


#### Questão 4
Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Educação no primeiro semestre de 2018.

In [12]:
q4_query = ('select ' +
    'mes_despesa, ano_despesa, programa_governo.nome_programa_governo, valor_despesa ' +
    'from despesa_publica ' + 
    'natural join orgao ' + 
    'natural join programa_governo ' + 
    'where nome_orgao=\'Ministério da Educação\' ' +
    'and ano_despesa=2018 ' +
    'and mes_despesa in (1,2,3,4,5,6)')
q4 = query(q4_query)
q4

,MES_DESPESA,ANO_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,1,2018,VIVER SEM LIMITE,0.0
1,2,2018,VIVER SEM LIMITE,0.0
2,3,2018,VIVER SEM LIMITE,0.0
3,4,2018,VIVER SEM LIMITE,0.0
4,5,2018,VIVER SEM LIMITE,0.0
...,...,...,...,...
148,2,2018,MAIS MEDICOS,0.0
149,3,2018,MAIS MEDICOS,0.0
150,4,2018,MAIS MEDICOS,0.0
151,5,2018,MAIS MEDICOS,0.0


#### Questão 5
Liste o código e nome dos órgãos que possuíram os 5 maiores valores de despesas totais nas funções de educação e saúde durante o ano de 2018.

In [13]:
q5_query = (
    'select cod_orgao, orgao.nome_orgao, sum(valor_despesa) as despesas_totais ' +
    'from despesa_publica ' +
    'natural join detalhamento_funcao ' +
    'natural join funcao ' +
    'natural join orgao ' +
    'where funcao.nome_funcao in (\'Saúde\', \'Educação\') ' +
    'group by cod_orgao order by despesas_totais desc'
)

q5 = query(q5_query)
q5

,COD_ORGAO,NOME_ORGAO,despesas_totais
0,52000,Ministério da Defesa,334126.70
1,26000,Ministério da Educação,64007.82
2,36000,Ministério da Saúde,50971.19
3,24000,"Ministério da Ciência, Tecnologia, Inovações",0.00
4,30000,Ministério da Justiça e Segurança Pública,0.00


#### Questão 6
Liste todos os nomes de funções e subfunções das despesas públicas dos últimos 5 meses de 2018 e seus respectivos valores totais, ordenando de forma decrescente pelo valor.

In [14]:
q6_query = (
    'select * from('
    'select nome_funcao, nome_subfuncao, sum(valor_despesa) as despesas_totais ' +
    'from despesa_publica ' +
    'natural join detalhamento_funcao ' +
    'natural join funcao ' +
    'natural join subfuncao ' +
    'where despesa_publica.ano_despesa = 2018 '
    'and despesa_publica.mes_despesa >= 8 '
    'group by nome_subfuncao, nome_funcao order by despesas_totais desc)'
    'where despesas_totais > 0'
)

q6 = query(q6_query)
q6

,nome_funcao,nome_subfuncao,despesas_totais
0,Educação,Educação básica,334126.70
1,Assistência social,Alimentação e nutrição,100373.55
2,Educação,Ensino profissional,7011.38
3,Educação,Ensino superior,2400.00
4,Saúde,Atenção básica,1000.00


#### Questão 12

In [15]:
q12_query = (
    'select distinct nome_funcao, valor_despesa '
    'from funcao '
    'natural join detalhamento_funcao '
    'natural join despesa_publica '
)

q12 = query(q12_query)
q12

,NOME_FUNCAO,VALOR_DESPESA
0,Essencial à justiça,0.00
1,Defesa nacional,0.00
2,Segurança pública,0.00
3,Assistência social,0.00
4,Assistência social,58918.28
5,Assistência social,8215.70
6,Assistência social,219668.07
7,Assistência social,3.42
8,Assistência social,19250.00
9,Assistência social,81120.13


#### Questão 13

In [6]:
q13_query = (
    'select distinct modalidade_despesa, valor_despesa, nome_orgao '
    'from orgao '
    'natural join despesa_publica '
    'where modalidade_despesa=\'Reserva de Contingência\' order by valor_despesa desc' # ordenado apenas para visualização
)

q13 = query(q13_query)
q13

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_ORGAO
0,Reserva de Contingência,334126.70,Ministério da Defesa
1,Reserva de Contingência,81120.13,"Ministério da Agricultura, Pecuária e Abastec"
2,Reserva de Contingência,58918.28,Ministério do Desenvolvimento Social
3,Reserva de Contingência,38971.19,Ministério da Saúde
4,Reserva de Contingência,28073.08,Ministério da Educação
5,Reserva de Contingência,19250.00,Ministério da Defesa
6,Reserva de Contingência,18178.00,Ministério da Educação
7,Reserva de Contingência,11000.00,Ministério da Saúde
8,Reserva de Contingência,8215.70,"Ministério da Agricultura, Pecuária e Abastec"
9,Reserva de Contingência,5900.00,Ministério da Educação


#### Questão 14

In [7]:
q14_query = (
    'select distinct valor_despesa, nome_programa_governo '
    'from despesa_publica '
    'natural join programa_governo '
    'where nome_programa_governo=\'ENEM\' order by valor_despesa desc' # Ordenado apenas para visualização
)

q14 = query(q14_query)
q14

,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,334126.70,ENEM
1,135.36,ENEM
2,0.00,ENEM


#### Questão 15

In [8]:
q15_query = (
    'select distinct nome_programa_governo, qtd_orgaos_subordinados, valor_despesa '
    'from despesa_publica '
    'natural join orgao '
    'natural join programa_governo '
    'where nome_orgao=\'Ministério da Educação\' order by valor_despesa desc' # Ordenado apenas para visualização
)

q15 = query(q15_query)
q15

,NOME_PROGRAMA_GOVERNO,QTD_ORGAOS_SUBORDINADOS,VALOR_DESPESA
0,PRONATEC,104,28073.08
1,PRONATEC,104,18178.00
2,PRONATEC,104,5900.00
3,PRONATEC,104,5000.00
4,MAIS MEDICOS,104,2400.00
5,PRONATEC,104,2000.00
6,REDE CEGONHA,104,1210.00
7,PRONATEC,104,726.00
8,PRONATEC,104,385.38
9,ENEM,104,135.36


---
### Escrevendo no .csv
```
#questao|#linhas|#colunas|sql-query
```

In [16]:
csv_df = pd.DataFrame(
    {
        '#questao':[1,2,3,4,5,6,12,13,14,15],
        '#linhas':[q1.shape[0], q2.shape[0], q3.shape[0], q4.shape[0], q5.shape[0], q6.shape[0], q12.shape[0], q13.shape[0], q14.shape[0], q15.shape[0]],
        '#colunas':[q1.shape[1], q2.shape[1], q3.shape[1], q4.shape[1], q5.shape[1], q6.shape[1], q12.shape[1], q13.shape[1], q14.shape[1], q15.shape[1]],
        'sql':[q1_last, q2_query, q3_query, q4_query, q5_query, q6_query, q12_query, q13_query, q14_query, q15_query],
    }
)

csv_df

,#questao,#linhas,#colunas,sql
0,1,3,3,"select funcao.nome_funcao, funcao.cod_funcao, ..."
1,2,4,3,"select * from(select funcao.nome_funcao, subfu..."
2,3,27,2,"select nome_programa_governo, sum(valor_despes..."
3,4,153,4,"select mes_despesa, ano_despesa, programa_gove..."
4,5,5,3,"select cod_orgao, orgao.nome_orgao, sum(valor_..."
5,6,5,3,"select * from(select nome_funcao, nome_subfunc..."
6,12,37,2,"select distinct nome_funcao, valor_despesa fro..."
7,13,32,3,"select distinct modalidade_despesa, valor_desp..."
8,14,3,2,"select distinct valor_despesa, nome_programa_g..."
9,15,26,3,"select distinct nome_programa_governo, qtd_org..."


In [17]:
csv_df.to_csv('2015042134.csv', sep='|')